In [21]:
import pandas as pd
import numpy as np
import re

In [46]:
# Load raw csv DEMOGRAPHICS
COUNTY_DEMOGRACHICS_CSV  = pd.read_csv('COUNTY_demographics.csv')
COUNTY_DEMOGRACHICS_CSV['county'].unique().shape

(3222,)

In [23]:
# Load raw csv LABOR 
COUNTY_LABOR_CSV  = pd.read_csv('COUNTY_Labor.csv')
COUNTY_LABOR_CSV.head(5)

,county,labor_force,employed,unemployed,unemployed_pct
0,"Autauga County, AL",25838,24576,1262,4.9
1,"Baldwin County, AL",96763,91338,5425,5.6
2,"Barbour County, AL",8587,7982,605,7.0
3,"Bibb County, AL",8640,8067,573,6.6
4,"Blount County, AL",24661,23653,1008,4.1


In [43]:
# Lookup dict, convert metro state abbreviation to full string
# We have abbreviations, database expects full state name
states = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
    
# invert the dictionary
states = dict(map(reversed, states.items()))


In [27]:
COUNTY_LABOR_CSV['abbr'] = ""
for index, row in COUNTY_LABOR_CSV.iterrows():
    try:
        locationParts = row['county'].split(', ')
        row['abbr']=locationParts[0] + ", "+states[locationParts[1]]
    except:
        print('nope', row)

nope county            District of Columbia
labor_force                     409734
employed                        376839
unemployed                       32895
unemployed_pct                8.0     
abbr                                  
Name: 319, dtype: object
nope county            NaN
labor_force       NaN
employed          NaN
unemployed        NaN
unemployed_pct    NaN
abbr                 
Name: 3219, dtype: object


In [28]:
COUNTY_LABOR_CSV['county'] = COUNTY_LABOR_CSV['abbr']

In [29]:
COUNTY_LABOR_CSV.drop(columns=['abbr'], inplace=True)

In [30]:
COUNTY_LABOR_CSV

,county,labor_force,employed,unemployed,unemployed_pct
0,"Autauga County, Alabama",25838,24576,1262,4.9
1,"Baldwin County, Alabama",96763,91338,5425,5.6
2,"Barbour County, Alabama",8587,7982,605,7.0
3,"Bibb County, Alabama",8640,8067,573,6.6
4,"Blount County, Alabama",24661,23653,1008,4.1
...,...,...,...,...,...
3215,"Vieques Municipio, Puerto Rico",N.A.,N.A.,N.A.,N.A.
3216,"Villalba Municipio, Puerto Rico",N.A.,N.A.,N.A.,N.A.
3217,"Yabucoa Municipio, Puerto Rico",N.A.,N.A.,N.A.,N.A.
3218,"Yauco Municipio, Puerto Rico",N.A.,N.A.,N.A.,N.A.


In [31]:
merged_df = pd.merge(COUNTY_LABOR_CSV, COUNTY_DEMOGRACHICS_CSV, on=['county'])

In [32]:
merged_df.dtypes

county              object
labor_force         object
employed            object
unemployed          object
unemployed_pct      object
population           int64
race_white           int64
race_black           int64
race_native          int64
race_asian           int64
race_islander        int64
race_other           int64
race_two_or_more     int64
dtype: object

In [37]:
merged_df.shape

(3256, 13)

In [51]:
# Drop rows where labor_force is null
merged_df = merged_df[pd.to_numeric(merged_df['labor_force'], errors='coerce').notnull()] 

In [52]:
merged_df.shape

(3194, 13)

In [53]:
merged_df['county'].unique().shape

(3127,)

In [54]:
merged_df.groupby(merged_df['county'],as_index=False).size()

,county,size
0,"Abbeville County, South Carolina",1
1,"Acadia Parish, Louisiana",1
2,"Accomack County, Virginia",1
3,"Ada County, Idaho",1
4,"Adair County, Iowa",1
...,...,...
3122,"Yuma County, Arizona",1
3123,"Yuma County, Colorado",1
3124,"Zapata County, Texas",1
3125,"Zavala County, Texas",1


In [59]:
merged_df = merged_df.drop_duplicates()

In [60]:
# write to csv
merged_df.to_csv('Employment_by_County.csv') 